In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.base import RegressorMixin
import joblib
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,mean_absolute_error, r2_score

In [10]:
# Load the preprocessed dataset
data = pd.read_csv(r'DataSets\New_Structured_Data\Model_TrainTest_data.csv')

In [11]:
data.head()

,oem,model,modelYear,Registration Year,Mileage,Fuel Type,Transmission,ownerNo,price,Gear Box,city,km,Safety,Interior,Exterior,Insurance Validity,bt
0,13,77,2015,2015.0,23.10,3,1,3,400000.0,5.0,0,120000.0,13.0,7.0,5.0,2,0
1,6,22,2018,2018.0,17.00,3,1,2,811000.0,5.0,0,32706.0,27.0,12.0,12.0,0,4
2,21,133,2018,2018.0,23.84,3,1,1,585000.0,5.0,0,11949.0,24.0,12.0,13.0,0,0
3,8,51,2014,2014.0,19.10,3,1,1,462000.0,5.0,0,17794.0,18.0,10.0,10.0,0,3
4,13,88,2015,2015.0,23.65,1,1,1,790000.0,5.0,0,60000.0,22.0,11.0,13.0,2,4


In [12]:
# Define the target variable
target = 'price'

# Split the data into features (X) and target (y)
X = data.drop(columns=[target])
y = data[target]

In [14]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize models (same as before)
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'ElasticNet': ElasticNet(),
    'Support Vector Regression (SVR)': SVR(),
    'K-Nearest Neighbors (KNN)': KNeighborsRegressor(),
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42)
}

# Store evaluation metrics for each model
evaluation_results = []

# Iterate over models
for name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Compute metrics
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Append metrics to results
    evaluation_results.append({
        'Model': name,
        'MAE': mae,
        'R² Score': r2
    })

# Convert results to a DataFrame for better readability
evaluation_df = pd.DataFrame(evaluation_results)

d:\DataScience\GUVI\DataScience_GUVI_Projects\guviproj_env\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.144e+14, tolerance: 8.857e+10
  model = cd_fast.enet_coordinate_descent(


# Model Evaluation Summary
- Based on the evaluation metrics (Mean Absolute Error (MAE) and R² Score):
  - **Random Forest** and **XGBoost** outperformed other models:
    - Random Forest: MAE = 70,561.81, R² = 0.9017
    - XGBoost: MAE = 60,683.88, R² = 0.9420
  - Both models demonstrated significantly better accuracy and lower error compared to other algorithms.

# Final Decision
- The **Random Forest** and **XGBoost** models will be taken to the next stage of refinement.
- **Hyperparameter Tuning**:
  - Utilize `RandomizedSearchCV` to optimize the performance of these two models.
  - This approach ensures efficient search across a wide range of hyperparameters while saving computational time compared to `GridSearchCV`.

In [15]:
evaluation_df

,Model,MAE,R² Score
0,Linear Regression,137459.206716,0.704059
1,Ridge Regression,137451.812927,0.704054
2,Lasso Regression,137457.420919,0.704061
3,ElasticNet,142626.774309,0.665231
4,Support Vector Regression (SVR),283890.140574,-0.069013
5,K-Nearest Neighbors (KNN),243771.203777,0.211243
6,Random Forest,70561.808188,0.901698
7,Gradient Boosting,87359.351706,0.892084
8,XGBoost,60683.883507,0.942015


# HyperParameter Tuning

In [16]:
# Define parameter grid for Random Forest
rf_param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Initialize Random Forest model
rf_model = RandomForestRegressor(random_state=42)

# Randomized Search CV
rf_random_search = RandomizedSearchCV(
    estimator=rf_model,
    param_distributions=rf_param_grid,
    n_iter=50,  # Number of random combinations to try
    cv=5,
    scoring='neg_mean_absolute_error',
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Fit the RandomizedSearchCV
rf_random_search.fit(X_train, y_train)

# Best parameters and model
rf_best_model = rf_random_search.best_estimator_
print("Best Parameters for Random Forest:", rf_random_search.best_params_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Parameters for Random Forest: {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': None, 'bootstrap': True}


In [17]:
# Random Forest Evaluation
rf_y_pred = rf_best_model.predict(X_test)
rf_mae = mean_absolute_error(y_test, rf_y_pred)
rf_r2 = rf_best_model.score(X_test, y_test)
print(f"Random Forest - MAE: {rf_mae:.2f}, R² Score: {rf_r2}")

Random Forest - MAE: 69497.43, R² Score: 0.9074309809122305


In [18]:
# Define parameter grid for XGBoost
xgb_param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 10],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 1, 5],
    'reg_alpha': [0, 0.5, 1],
    'reg_lambda': [1, 2, 5]
}

# Initialize XGBoost model
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)

# Randomized Search CV
xgb_random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=xgb_param_grid,
    n_iter=50,  # Number of random combinations to try
    cv=5,
    scoring='neg_mean_absolute_error',
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Fit the RandomizedSearchCV
xgb_random_search.fit(X_train, y_train)

# Best parameters and model
xgb_best_model = xgb_random_search.best_estimator_
print("Best Parameters for XGBoost:", xgb_random_search.best_params_)


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Parameters for XGBoost: {'subsample': 1.0, 'reg_lambda': 2, 'reg_alpha': 1, 'n_estimators': 500, 'max_depth': 7, 'learning_rate': 0.1, 'gamma': 0, 'colsample_bytree': 0.6}


In [19]:
# XGBoosting Evaluation
xgb_y_pred = xgb_best_model.predict(X_test)
xgb_mae = mean_absolute_error(y_test, xgb_y_pred)
xgb_r2 = xgb_best_model.score(X_test, y_test)
print(f"XGBoosting - MAE: {xgb_mae:.2f}, R² Score: {xgb_r2}")

XGBoosting - MAE: 56519.18, R² Score: 0.949466415285093


In [22]:
results = {
    'Model': ['Random Forest', 'XGBoost'],
    'MAE': [rf_mae, xgb_mae],
    'R² Score': [rf_r2, xgb_r2]
}
results_df = pd.DataFrame(results)
print(results_df)

           Model           MAE  R² Score
0  Random Forest  69497.433437  0.907431
1        XGBoost  56519.176737  0.949466


In [24]:
# Save the best model (XGBoost)
joblib.dump(xgb_best_model, 'xgboost_best_model.pkl')
print("Model saved successfully as 'xgboost_best_model.pkl'")

Model saved successfully as 'xgboost_best_model.pkl'
